# Toronto Dwellings Analysis Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions to create a Panel dashboard.

In [27]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [28]:
# Initialize the Panel Extensions (for Plotly)
import panel as pn
pn.extension("plotly")

In [29]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("Mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [30]:
# Import the CSVs to Pandas DataFrames
file_path = Path("toronto_neighbourhoods_census_data.csv")
to_data = pd.read_csv(file_path, index_col="year")

file_path = Path("toronto_neighbourhoods_coordinates.csv")
df_neighbourhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

### Global available data

In [31]:
# Getting the data from the top 10 expensive neighbourhoods
# YOUR CODE HERE!
top_10 =to_data.groupby('neighbourhood')[['average_house_value']].mean().sort_values('average_house_value',ascending = False)[:10]
top_10

# Calculate the mean number of dwelling types units per year
# YOUR CODE HERE!
df_dwelling=to_data.groupby(['year']).mean()
df_dwelling
# Calculate the average monthly shelter costs for owned and rented dwellings
# YOUR CODE HERE!
shelter_rented_costs = to_data.groupby(['year'])[['shelter_costs_rented','shelter_costs_owned']].mean()
shelter_rented_costs

,shelter_costs_rented,shelter_costs_owned
year,,
2001,1085.935714,846.878571
2006,925.414286,1316.800000
2011,1019.792857,1448.214286
2016,1256.321429,1761.314286


### Panel Visualization Functions

In [32]:
# Define Panel visualization functions
def neighbourhood_map(df_neighbourhood, lat, lon, color):
    """Neighbourhood Map"""
    px.set_mapbox_access_token("pk.eyJ1IjoidGFudXNocmVlYXJvcmEiLCJhIjoiY2txNnQ3OTcxMDN4dzJ1cGNudGJoMXBzbCJ9.eS_tRDR900GQ4A63t9vB9A")
    fig = px.scatter_mapbox(df_neighbourhood, lat="lat", lon="lon",color="neighbourhood")
    fig.show()
    # YOUR CODE HERE!

def create_bar_chart(data1, i,title, xlabel, ylabel, color):
    """
    Create a barplot based in the data argument.
    """
    return data1.loc[i].hvplot.bar(title='BAR CHART FOR'+ str(i) ,xlabel=str(i),ylabel='Dwelling Type units',color='Red',rot=90).opts(yformatter='%.0f')
    

def create_line_chart(data, title, xlabel, ylabel, color):
    """
    Create a line chart based in the data argument.
    """
    fig = px.line(data, x=xlabel, y=ylabel, title=title)
    fig.update_traces(mode='markers+lines')
    fig.update_traces(line_color=color)
    fig.show()


def average_house_value(data):
    """Average house values per year."""
    data.groupby('year')[['average_house_value']].mean()
    return data
   

def average_value_by_neighbourhood(data1, neighbourhood):
    """Average house values by neighbourhood."""
    data = data1[data1['neighbourhood']== neighbourhood]
    def average_house_value(data):
        """Average house values per year."""
        return data.groupby('year')[['average_house_value']].mean()
    return average_house_value(data)
    

def number_dwelling_types(data):
    """Number of dwelling types per year"""
    return data.groupby(['year']).sum()
    

def average_house_value_snapshot(data):
    """Average house value for all Toronto's neighbourhoods per year."""
    return data.groupby(['year'])[['average_house_value']].mean()
    

def top_most_expensive_neighbourhoods(data):
    """Top 10 most expensive neighbourhoods."""
    return data.groupby('neighbourhood')[['average_house_value']].mean().sort_values('average_house_value',ascending = False)[:10]
    

def sunburts_cost_analysis(data):
    """Sunburst chart to conduct a costs analysis of most expensive neighbourhoods in Toronto per year."""
    df = data.groupby('neighbourhood')[['average_house_value']].mean().sort_values('average_house_value',ascending = False)[:10].reset_index()
  
    fig = px.sunburst(df, path=['neighbourhood'], 
                      values='average_house_value', color='average_house_value')
    fig.show()


## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [33]:
# create widget
select = pn.widgets.Select(name='Select', options=list(to_data.neighbourhood.values))


def average_value_by_neighbourhood(symbol):
    """Average house values by neighbourhood."""
    data = to_data[to_data['neighbourhood']== symbol]
    def average_house_value(data):
        """Average house values per year."""
        return data.groupby('year')[['average_house_value']].mean()
    return average_house_value(data).reset_index().hvplot('year', 'average_house_value')


layout = pn.Column(
    pn.Column(select),
    average_value_by_neighbourhood(select.options[0])
)

# callback
def update(event):
    # layout[1] is chart element, layout[0] is dropdown select
    layout[1].object = average_value_by_neighbourhood(select.value)
    
# update or replace the already rendered components when a watched parameter changes
select.param.watch(update, 'value')

# returns the panel object to allow it to display itself in a notebook
layout.servable()


Column
    [0] Column
        [0] Select(options=['Agincourt North', ...], value='Agincourt North')
    [1] HoloViews(Curve)

In [37]:
# Create a Title for the Dashboard
# YOUR CODE HERE!
pn.panel('<marquee> Drizzling Insights </marquee>')
# Define a welcome text
pn.pane.Str ('Welcome to my analytics page')

# Create a tab layout for the dashboard
# YOUR CODE HERE!
# create widget
select = pn.widgets.Select(name='Select', options=list(to_data.neighbourhood.values))


def average_value_by_neighbourhood(symbol):
    """Average house values by neighbourhood."""
    data = to_data[to_data['neighbourhood']== symbol]
    def average_house_value(data):
        """Average house values per year."""
        return data.groupby('year')[['average_house_value']].mean()
    return average_house_value(data).reset_index().hvplot('year', 'average_house_value')


# returns the panel object to allow it to display itself in a notebook
dashboard = pn.Column('## Drizzling Insights',
                   '## Welcome to my analytics page',
    pn.Column(select),
    average_value_by_neighbourhood(select.options[0])
)

# callback
def update(event):
    # dashboard[1] is chart element, dashboard[0] is dropdown select
    dashboard[1].object = average_value_by_neighbourhood(select.value)
    
# update or replace the already rendered components when a watched parameter changes
select.param.watch(update, 'value')

# returns the panel object to allow it to display itself in a notebook
dashboard.servable()


Column
    [0] Markdown(str)
    [1] Markdown(str)
    [2] Column
        [0] Select(options=['Agincourt North', ...], value='Agincourt North')
    [3] HoloViews(Curve)

## Serve the Panel Dashboard

In [35]:
dashboard.servable()

Column
    [0] Markdown(str)
    [1] Markdown(str)
    [2] Column
        [0] Select(options=['Agincourt North', ...], value='Agincourt North')
    [3] HoloViews(Curve)